In [1]:
import nibabel as nib
from dipy.align.reslice import reslice
from pathlib import Path, WindowsPath, PureWindowsPath, PosixPath, PurePosixPath
from tqdm import tqdm
import os
from typing import Tuple
import numpy as np
from skimage.transform import resize

In [2]:
#f_string = "C:\\Users\\Moritz\\repositories\\dataset\\Task07_Pancreas"
f_string = "/Users/moritzknolle/Desktop/Uni/deep_learning_adenocarcinoma/Data/Task07_Pancreas"
data_path = Path(f_string)

In [3]:
img_path = data_path / "imagesTr"
label_path = data_path / "labelsTr"

In [4]:
assert img_path.exists() and label_path.exists()

In [5]:
# loading scans
scan_names = [file for file in img_path.rglob("*.nii.*")]
scans = [nib.load(str(scan_name)) for scan_name in scan_names]

In [6]:
# loading labels
label_names = [file for file in label_path.rglob("*.nii.*")]
labels = [nib.load(str(scan_name)) for scan_name in label_names]

In [7]:
assert len(labels) == len(scans)

In [8]:
if not os.path.exists(data_path / "images_resampled") and not os.path.exists(data_path / "labels_resampled"):
    imgs_path = data_path / "images_resampled"
    lbls_path = data_path / "labels_resampled"
    imgs_path.mkdir()
    lbls_path.mkdir()

In [9]:
NEW_ZOOMS = (2., 2., 2.)

In [13]:
def get_name(nifti:nib.Nifti1Image)->str:
    "Gets the original filename of a Nifti."
    file_dir = nifti.get_filename()
    if isinstance(data_path, PosixPath) or isinstance(data_path, PurePosixPath):
        file_name = file_dir.split("/")[-1]
    else:
        file_name = file_dir.split("\\")[-1]
    return file_name

In [14]:
def resample_nii(nifti:nib.Nifti1Image, new_pix_dims:Tuple[float,...], num_workers:int) -> Tuple[np.ndarray, ...]:
    "Resamples nifti pixels to new_pix_dims and returns the new array and new affine using num_workers threads."
    return reslice(nifti.get_fdata(), nifti.affine, nifti.header.get_zooms(), new_pix_dims, num_processes=num_workers)

In [15]:
def resize_array(ar:np.ndarray, out_size:Tuple[int,...], isLabel:bool) -> np.ndarray:
    "Rescales ar to out_size with nearest neighbour and maintains value ranges."
    return resize(ar, out_size, preserve_range=True, order = 0)

In [17]:
destination_path = data_path / "resampling_experiment" / "resampled_2x2x2"
i=0
for scan, label in zip(scans, labels):
    i+=1
    print("... resampling scan no:", i, get_name(scan))
    resampled_vol, new_vol_affine = resample_nii(scan, NEW_ZOOMS, 0)
    resampled_mask, new_mask_affine = resample_nii(label, NEW_ZOOMS, 0)

    resized_vol = resize_array(resampled_vol, (512, 512, resampled_vol.shape[2]), False)
    resized_mask = resize_array(resampled_mask, (512, 512, resampled_mask.shape[2]), True)

    vol_resliced = nib.Nifti1Image(resized_vol, new_vol_affine)
    mask_resliced = nib.Nifti1Image(resized_mask, new_mask_affine)

    vol_path = destination_path / "imagesTr" / get_name(scan)
    mask_path = destination_path / "labelsTr" / get_name(label)
    nib.save(vol_resliced, str(vol_path))
    nib.save(mask_resliced, str(mask_path))
print("done")

... resampling scan no: 1 pancreas_290.nii.gz
... resampling scan no: 2 pancreas_127.nii.gz
... resampling scan no: 3 pancreas_056.nii.gz
... resampling scan no: 4 pancreas_148.nii.gz
... resampling scan no: 5 pancreas_048.nii.gz
... resampling scan no: 6 pancreas_135.nii.gz
... resampling scan no: 7 pancreas_382.nii.gz
... resampling scan no: 8 pancreas_035.nii.gz
... resampling scan no: 9 pancreas_160.nii.gz
... resampling scan no: 10 pancreas_103.nii.gz
... resampling scan no: 11 pancreas_172.nii.gz
... resampling scan no: 12 pancreas_111.nii.gz
... resampling scan no: 13 pancreas_212.nii.gz
... resampling scan no: 14 pancreas_312.nii.gz
... resampling scan no: 15 pancreas_300.nii.gz
... resampling scan no: 16 pancreas_200.nii.gz
... resampling scan no: 17 pancreas_355.nii.gz
... resampling scan no: 18 pancreas_404.nii.gz
... resampling scan no: 19 pancreas_255.nii.gz
... resampling scan no: 20 pancreas_328.nii.gz
... resampling scan no: 21 pancreas_228.nii.gz
... resampling scan no